In [1]:
#!pip3 install ray torch torchvision tabulate tensorboard
#!pip3 install 'ray[rllib]'
#!pip3 install ray

In [2]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

from models import VisualEncoder
from train import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        features_dim = 512
        self.encoder = VisualEncoder(features_dim)
        self.encoder.load_state_dict(
            torch.load("/src/Visual Autoencoder weights and models/IGLU_encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.action_head = nn.Linear(features_dim, action_space.n)
        self.value_head = nn.Linear(features_dim, 1)
        self.last_value = None
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.encoder.cuda()
            self.action_head.cuda()
            self.value_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs'].permute(0, 3, 1, 2).float() / 255.0
        if self.use_cuda:
            obs.cuda()
            
        features = self.encoder(obs)
        action = self.action_head(features)
        self.last_value = self.value_head(features).squeeze(1)
        return action, state
    
    @override(TorchModelV2)
    def value_function(self):
        assert self.last_value is not None, "must call forward() first"
        return self.last_value

In [4]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=1000)
    env.update_taskset(TaskSet(preset=['C22']))
    env = PovOnlyWrapper(env)
    env = IgluActionWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

In [ ]:
from ray.tune.integration.wandb import WandbLogger

tune.run(PPOTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 1,
             "sgd_minibatch_size": 256,
             "clip_param": 0.2,
             "entropy_coeff": 0.01,
             "lambda": 0.95,
             "train_batch_size": 1000,
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "PPO C22 pretrained"
                  }
              }

        },
        loggers=[WandbLogger])#callbacks=[
        #    CustomLoggerCallback(),
        #])

/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


Trial name,status,loc
PPO_my_env_ccae9_00000,PENDING,


2021-09-17 08:54:00,356	INFO wandb.py:170 -- Already logged into W&B.
wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
wandb: WARNING Tried to auto resume run with id f02b1_00000 but id ccae9_00000 is set.
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
(pid=177) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=177)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=177) 2021-09-17 08:54:06,313	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=177) 2021-09-17 08:54:06,313	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(pid=163) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=163)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=177) 2021-09-17 08:54:32,399	INFO trainable.py:109 -- Trainable.setup took 30.337 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(pid=177) 2021-09-17 08:54:32,400	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(pid=163) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=163)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=163) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=163)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=163) 

Result for PPO_my_env_ccae9_00000:
  agent_timesteps_total: 1000
  custom_metrics: {}
  date: 2021-09-17_08-56-47
  done: false
  episode_len_mean: 1000.0
  episode_media: {}
  episode_reward_max: 2.0
  episode_reward_mean: 2.0
  episode_reward_min: 2.0
  episodes_this_iter: 1
  episodes_total: 1
  experiment_id: 92f384b1516d4da09515d85b259ef30e
  hostname: cdsserver
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.19999999999999996
          cur_lr: 5.000000000000001e-05
          entropy: 1.2638701505131191
          entropy_coeff: 0.009999999999999998
          kl: 0.007371294528260108
          policy_loss: -0.14228328110443222
          total_loss: -0.055882537023474774
          vf_explained_var: -0.2014622986316681
          vf_loss: 0.09756518351948923
    num_agent_steps_sampled: 1000
    num_agent_steps_trained: 1000
    num_steps_sampled: 1000
    num_steps_trained: 1000
  iterations_since_restore: 1
  node_ip: 192.168.1.96
  num_he

...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_ccae9_00000,RUNNING,192.168.1.96:177,64,1091.07,64000,0.25,3,-5,996.094


Result for PPO_my_env_ccae9_00000:
  agent_timesteps_total: 65000
  custom_metrics: {}
  date: 2021-09-17_09-13-00
  done: false
  episode_len_mean: 996.1538461538462
  episode_media: {}
  episode_reward_max: 3.0
  episode_reward_mean: 0.24615384615384617
  episode_reward_min: -5.0
  episodes_this_iter: 1
  episodes_total: 65
  experiment_id: 92f384b1516d4da09515d85b259ef30e
  hostname: cdsserver
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.09621691927313802
          cur_lr: 5.000000000000001e-05
          entropy: 2.454525521066454
          entropy_coeff: 0.009999999999999998
          kl: 0.016052547004071768
          policy_loss: -0.020937303288115397
          total_loss: 0.0569317452609539
          vf_explained_var: -0.11801651120185852
          vf_loss: 0.10086977941294512
    num_agent_steps_sampled: 65000
    num_agent_steps_trained: 65000
    num_steps_sampled: 65000
    num_steps_trained: 65000
  iterations_since_restore: 65

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_ccae9_00000,RUNNING,192.168.1.96:177,65,1105.96,65000,0.246154,3,-5,996.154


Result for PPO_my_env_ccae9_00000:
  agent_timesteps_total: 66000
  custom_metrics: {}
  date: 2021-09-17_09-13-15
  done: false
  episode_len_mean: 996.2121212121212
  episode_media: {}
  episode_reward_max: 3.0
  episode_reward_mean: 0.2727272727272727
  episode_reward_min: -5.0
  episodes_this_iter: 1
  episodes_total: 66
  experiment_id: 92f384b1516d4da09515d85b259ef30e
  hostname: cdsserver
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.09621691927313802
          cur_lr: 5.000000000000001e-05
          entropy: 2.242608322037591
          entropy_coeff: 0.009999999999999998
          kl: 0.02377127594185347
          policy_loss: 0.04443291384312842
          total_loss: 0.059659829032089974
          vf_explained_var: 0.1345871090888977
          vf_loss: 0.035365797069648076
    num_agent_steps_sampled: 66000
    num_agent_steps_trained: 66000
    num_steps_sampled: 66000
    num_steps_trained: 66000
  iterations_since_restore: 66
  

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_ccae9_00000,RUNNING,192.168.1.96:177,66,1120.82,66000,0.272727,3,-5,996.212
